# Harmony integration for scIB comparison to scVI

In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import scib

from scib_metrics.benchmark import Benchmarker

sc.logging.print_header()

/home/jovyan/my-conda-envs/scib_harmony/lib/python3.8/site-packages/numba/core/ir_utils.py:1525: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if (hasattr(numpy, value)
/home/jovyan/my-conda-envs/scib_harmony/lib/python3.8/site-packages/numba/core/ir_utils.py:1526: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  and def_val == getattr(numpy

scanpy==1.7.1 anndata==0.8.0 umap==0.4.6 numpy==1.20.1 scipy==1.10.1 pandas==1.2.3 scikit-learn==0.24.1 statsmodels==0.12.2 python-igraph==0.8.3 louvain==0.7.0 leidenalg==0.8.3


In [2]:
adata = sc.read_h5ad('/nfs/team205/ao15/Megagut/Annotations_v3/h5ad/scib_benchmarking/pooled_healthy.gene_cellbender.good_qc_cluster_mito80.doublet_flagged.20220909.finalannot_unintegrated.scib20231108.h5ad')

/home/jovyan/my-conda-envs/scib_harmony/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
adata_hvg = sc.read_h5ad('/nfs/team205/ao15/Megagut/Annotations_v3/models/scvi_output20230126/pooled_healthy.hvg7500_noCC.nodoublets.scvi_output.20230126-ad1.h5ad')

/home/jovyan/my-conda-envs/scib_harmony/lib/python3.8/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)


In [4]:
adata = adata[:, adata.var_names.isin(adata_hvg.var_names)].copy()

In [6]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [7]:
sc.tl.pca(adata, n_comps=30, use_highly_variable=False)

In [8]:
from harmony import harmonize
Z = harmonize(adata.obsm['X_pca'], adata.obs, batch_key = 'donorID_unified')
adata.obsm['Harmony'] = Z

	Initialization is completed.
	Completed 1 / 10 iteration(s).
	Completed 2 / 10 iteration(s).
	Completed 3 / 10 iteration(s).
	Completed 4 / 10 iteration(s).
	Completed 5 / 10 iteration(s).
Reach convergence after 5 iteration(s).


In [9]:
pd.DataFrame(
    adata.obsm["Harmony"],
    index=adata.obs_names,
    columns=[f"LV{i+1}" for i in range(adata.obsm["Harmony"].shape[1])],
).to_csv(
    "/nfs/team205/ao15/Megagut/Annotations_v3/healthy_analysis/integration_benchmarking/fulladata_embeddings/Harmony.csv.gz"
)

/home/jovyan/my-conda-envs/scib_harmony/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
harmony_int = pd.read_csv('/nfs/team205/ao15/Megagut/Annotations_v3/healthy_analysis/integration_benchmarking/fulladata_embeddings/Harmony.csv.gz',index_col=0,compression='gzip')

In [11]:
harmony_int

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV21,LV22,LV23,LV24,LV25,LV26,LV27,LV28,LV29,LV30
index,,,,,,,,,,,,,,,,,,,,,
AAACGAATCTCTTGCG-GSM4600896,6.792508,1.493446,2.270899,7.442399,23.390062,-6.418595,-4.714101,4.415015,-5.116053,0.398806,...,-1.273105,-0.170802,0.704225,-1.149834,-0.399836,-0.235145,-1.776230,-1.551711,0.236224,-0.085833
AAACGCTAGCGACCCT-GSM4600896,-4.457764,-12.781011,-0.002760,1.874083,-3.824457,-5.576455,-0.228090,2.598805,1.421712,0.887581,...,-0.152506,0.495729,0.903175,-2.073839,3.006745,1.290129,-0.144248,-0.658522,-2.557939,-1.280467
AAAGAACAGCGACCCT-GSM4600896,7.264336,1.749831,1.610741,7.703456,20.231460,-2.257334,-6.510646,4.415112,-6.450018,-0.251699,...,0.195840,-0.291711,0.553728,-0.077108,1.159765,1.896198,-0.551654,-1.124575,-1.157414,3.607705
AAAGGGCAGTCACACT-GSM4600896,-1.577039,-9.288579,-1.272927,-4.578127,1.224974,-1.675923,-1.970577,1.783844,-2.548688,-1.299638,...,-1.753042,0.516714,1.091329,-1.524200,0.867595,1.285773,0.666479,-0.742319,-3.153852,0.406093
AAAGGGCCAACACAGG-GSM4600896,-3.690262,-11.402837,-2.912959,5.151815,6.655346,11.707860,1.239655,-4.840766,1.602466,1.775268,...,-1.443013,-1.479720,0.722223,-2.157584,-0.731456,-1.280247,1.547629,0.639453,1.009290,1.930844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCAAGAACAACT-GSM5525958,-3.569559,-14.521702,-0.728541,9.731349,-4.850459,-4.275488,-0.276917,2.789367,0.958148,1.780673,...,1.504967,0.389203,-0.603116,-0.847376,-0.383137,2.215331,-0.258343,-0.210507,-0.352328,0.709538
TTTGTCAAGATAGGAG-GSM5525958,-2.292066,-6.689699,0.931712,-8.444520,-0.681993,1.825850,-1.996322,4.578505,0.108327,-1.475057,...,0.237004,-0.248287,-0.406984,0.726705,0.485472,0.606396,-0.165669,0.164016,-1.265021,-0.466313
TTTGTCAAGTAGGCCA-GSM5525958,-2.282250,-7.273726,0.925961,-6.974571,-0.841373,2.289356,-2.070351,4.415245,0.230755,-1.366043,...,0.416104,-0.424804,-0.520070,0.575418,-0.306826,0.534763,-0.295746,-1.569897,-0.639689,-0.950438
